In [1]:
import os
import nltk
from pymorphy2 import MorphAnalyzer
m = MorphAnalyzer()
from nltk.tokenize import RegexpTokenizer
from pymorphy2.tokenizers import simple_word_tokenize
tokenizer = RegexpTokenizer(r'\w+')

import RAKE
nltk.download('stopwords')
from nltk.corpus import stopwords
stop = stopwords.words('russian')
rake = RAKE.Rake(stop)
from summa import keywords
import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.summarization import keywords as kw


def normalize_text(text):
    text = tokenizer.tokenize(text.lower())
    lemmas = [m.parse(t)[0].normal_form for t in text]
    return ' '.join(lemmas)


def normalize(text):
    text = tokenizer.tokenize(text.lower())
    return [m.parse(t)[0].normal_form for t in text]

[nltk_data] Error loading stopwords: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>


In [2]:
with open('./texts/meta.txt', 'r', encoding='utf-8') as f:
    meta = [normalize(line) for line in f.read().split('\n')]
meta

[['индивидуальный',
  'различие',
  'характеристика',
  'признак',
  'психология',
  'причина',
  'личность'],
 ['психологический',
  'тип',
  'типология',
  'характеристика',
  'различие',
  'группа',
  'характер'],
 ['черта',
  'психологический',
  'характеристика',
  'отличие',
  'концептуализация',
  'свойство',
  'классификация'],
 ['интеллект',
  'исследование',
  'тип',
  'показатель',
  'информация',
  'память',
  'характеристика'],
 ['личность',
  'темперамент',
  'эмоциональность',
  'поведение',
  'характеристика',
  'тип',
  'характер']]

In [3]:
data = []
for file in os.listdir('./texts'):
    if file.startswith('text'):
        path = os.path.join('./texts/', file)
        with open(path, 'r', encoding='utf-8') as f:
            nb = int(file[4])
            data.append(tuple([f.read(), meta[nb-1]]))

corpus = [d[0] for d in data]       

In [4]:
data = []
for file in os.listdir('./texts'):
    if file.startswith('text'):
        path = os.path.join('./texts/', file)
        with open(path, 'r', encoding='utf-8') as f:
            data.append(tuple([f.read(), meta[nb-1]]))

In [5]:
def textRank(text):
    text = normalize_text(text)
    result = []
    kw(text, pos_filter=[], scores=True)
    keywords.keywords(normalize_text(text), additional_stopwords=stop, scores=True)
    G = keywords.get_graph(text)
    result = [(edge, G.edge_weight(edge)) for edge in G.edges() if all(map(lambda x: x not in stop, edge))]
    result = sorted(result, key = lambda x: x[1], reverse=True)[:7]
    for i in result:
        print(i)
    return result

In [6]:
 def TfIdf(text, ng = (1,3)):
    vectorizer = TfidfVectorizer(ngram_range = ng)
    X = vectorizer.fit_transform([normalize_text(text)])
    names = vectorizer.get_feature_names()
    words = {}
    for col in X.nonzero()[1]:
        if names[col] not in stop:
            words[names[col]] =  float(X[0, col])
    top = sorted(words.items(), key = lambda x: x[1], reverse=True)
    for i in top[:5]:
        print(i[0], i[1])
    return [i[0] for i in top[:7]]

In [7]:
def Rake(text):
    pred = []
    for i in rake.run(normalize_text(text), minFrequency=2, maxWords=3)[:7]:
            print(i[0], i[1])
            pred.append(i[0])
    return pred

In [ ]:
textrank_res = []
for d in corpus:
    textrank_res.append(textRank(d))
    print('============')

In [ ]:
tfidf_res = []
for d in corpus:
    tfidf_res.append(TfIdf(d, ng = (3,3)))
    print('==================')

In [ ]:
for d in corpus:
    TfIdf(d, ng = (1,3))
    print('==================')

In [ ]:
tfidf_res = []
for d in corpus:
    tfidf_res.append(TfIdf(d, ng = (2,2)))
    print('==================')

In [ ]:
rake_res = []
for d in data:
    rake_res.append(Rake(d[0]))
    print('==================')

In [ ]:
print(tfidf_res, '\n', textrank_res, '\n', rake_res)

In [ ]:
from sklearn.metrics import f1_score
metrics_true = []
y_pred = []
for line in meta:
    metrics_true.append([1] * len(line))
    y_pred.append([0] * len(line))
(metrics_true, y_pred)

In [ ]:
from copy import deepcopy

In [ ]:
rake_pred = deepcopy(y_pred)
tfidf_pred = deepcopy(y_pred)
textrank_pred = deepcopy(y_pred)

### TfIdf

In [ ]:
for nb,text in enumerate(tfidf_res):
    print(nb, text, meta[nb])
    for idx, words in enumerate(text):
        line = words.split()
        tfidf_pred[nb][idx] = int(True in (map(lambda x: x in meta[nb], line)))
tfidf_pred            

In [ ]:
y_true = [item for inner in metrics_true for item in inner]
y_pred = [item for inner in tfidf_pred for item in inner]
# sklearn.metrics.accuracy_score(y_true, y_pred), sklearn.metrics.recall_score(y_true, y_pred), f1_score(y_true, y_pred) 
m = sklearn.metrics.precision_recall_fscore_support(y_true, y_pred)
print('\t'.join([str(item) for item in m]))

#### Ошибки

* Не выкидывает местоимения (какой-то), нужно их включать в стоп-слова
* Самые частотные != ключевые

### Rake

In [ ]:
for nb,text in enumerate(rake_res):
    print(nb, text, meta[nb])
    for idx, words in enumerate(text):
        line = words.split()
        rake_pred[nb][idx] = int(True in (map(lambda x: x in meta[nb], line)))
rake_pred  

In [ ]:
#RAKE
y_pred = [item for inner in rake_pred for item in inner]

m = sklearn.metrics.precision_recall_fscore_support(y_true, y_pred)
print('\t'.join([str(item) for item in m]))

##### Ошибки

* Лишние слова, которые берет во внимание - "который", "е"
* В биграммах и просто словах - повторение одних и тех же слов ("выше интеллект" и "интеллект")
* Самыйй частотные слова != ключевые

### TextRank

In [ ]:
for nb,text in enumerate(textrank_res):
    for idx, words in enumerate(text):
        textrank_pred[nb][idx] = int(True in (map(lambda x: x in meta[nb], words)))
textrank_pred  

In [ ]:
#text_rank
y_pred = [item for inner in textrank_pred for item in inner]
# sklearn.metrics.accuracy_score(y_true, y_pred), sklearn.metrics.recall_score(y_true, y_pred), f1_score(y_true, y_pred) \
m = sklearn.metrics.precision_recall_fscore_support(y_true, y_pred)
print('\t'.join([str(item) for item in m]))

#### Ошибки:

* Считает биграммы подряд, в том порядке, в котором они в тексте
* В данном случае для всех биграмм частота равна 1, т.е. они встретилсь в тексте один раз, что сомнительно
* В итоговом результате - ни одного пересечения с ключевыми словами



### Итог

* Для поиска ключевых слов недостаточно знать частотность
* В исходных текстах встречалось много примеров описывамых явлений, поэтому частостность ключевых слов уменьшалась засчет преобладания других слов
* Системы не всегда считают частоты биграмм корректно, включая повторы слов или лишние слова